# Introduction to Using Spring AI with Kotlin

This notebook provides an introductory tutorial on using Spring AI in Kotlin to interact with large language models, using an OpenAI example. We'll go through the process step by step to configure it, use prompts, handle streaming responses, obtain structured data as a result and use tools.

### Setting Up Your Project

Make sure your project is set up to include the Spring AI dependencies:

In [ ]:
USE {
    repositories {
        mavenCentral()
        maven(url = "https://repo.spring.io/milestone")
        maven(url = "https://repo.spring.io/snapshot")
    }

    dependencies {
        val springAiVersion = "1.0.0-M5"
        implementation("org.springframework.ai:spring-ai-openai:$springAiVersion")
        implementation("org.springframework.ai:spring-ai-openai-spring-boot-starter:$springAiVersion")
        implementation("com.fasterxml.jackson.module:jackson-module-kotlin:2.18.2")
    }
}

Provide your OpenAI API key by setting up an environmental variable `OPENAI_API_KEY`. Alternatively, you can copy it here:

In [ ]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

Set up the OpenAI chat model with your API key and specify the desired configuration (such as temperature and model type):

In [ ]:
import org.springframework.ai.openai.OpenAiChatModel
import org.springframework.ai.openai.OpenAiChatOptions
import org.springframework.ai.openai.api.OpenAiApi

val openAiApi = OpenAiApi(apiKey)
val openAiChatOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.3)
    .build()
val chatModel = OpenAiChatModel(openAiApi, openAiChatOptions)

### Sending Prompts

Interact with the API by sending a simple prompt to the chat model and receiving a response:

In [ ]:
chatModel.call("Generate a hokku about Kotlin")

Use Spring AI's `ChatClient` to define more complex prompts, such as providing system instructions:

In [ ]:
import org.springframework.ai.chat.client.ChatClient

val chatClient = ChatClient.builder(chatModel).defaultSystem(
    """
    You are a Lord of the Rings expert and a trusted advisor.
    Offer wise, concise guidance in the style of Middle-earth,
    drawing from its lore, characters, and philosophy.
    """.trimIndent()
).build()

Now you can send a user-defined prompt to a chat model and retrieve the response content as `String`:

In [ ]:
chatClient
    .prompt()
    .user("What awaits us?")
    .call()
    .content()


Try replacing the `content()` call with `chatResponse()` to gain more insight into the response. `ChatResponse` represents the AI model's response and includes metadata on how it was generated, such as the number of tokens used.

### Handling Streaming Responses

Using the `stream()` method you get unfinished chunks of response when they are ready. This way, you don't wait for the AI to generate an entire response, and can show real-time progress to users.


Include the dependency on coroutines to work with the result as a Kotlin `Flow`:

In [ ]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactor:1.10.1")

In a reactive UI, you can show the incoming response in real time. To keep this example simple, we display different chunks of the response, each on a separate line (though they are printed simultaneously):

In [ ]:
import kotlinx.coroutines.reactive.asFlow

val streamingResponse: Flow<String> = chatModel
    .stream("Generate a hokku about Kotlin")
    .asFlow()

runBlocking {
    streamingResponse.collect {
        println(it)
    }
}


Since `collect` is a suspend function, we surround it with the `runBlocking` call to use it in a notebook.

### Structured Output

Spring AI allows you to deserialize the responses into Kotlin data classes automatically, making it easy to handle structured outputs.

Let's get the response from LLM about the movie in the desired format:

In [ ]:
data class Movie(
    val title: String,
    val year: Int,
    val director: String,
    val genre: String
)

Specify `ResponseFormat` as `JSON_OBJECT` to instruct LLM to return the output strictly in JSON and Spring AI to automatically convert it to a `data` class:

In [ ]:
import org.springframework.ai.openai.api.ResponseFormat

val structuredOutputOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O)
    .responseFormat(ResponseFormat.builder().type(ResponseFormat.Type.JSON_OBJECT).build())
    .build()
val chatModelWithStructuredOutput = OpenAiChatModel(openAiApi, structuredOutputOptions)

In the following example, OpenAI returns the required JSON, which gets automatically converted to a `Movie`:

In [ ]:
ChatClient.create(chatModelWithStructuredOutput)
    .prompt()
    .user("Movie that won the Oscar for Best Picture in 1990")
    .call()
    .entity(Movie::class.java)

As AI models often hallucinate and are not guaranteed to return the correct answer, they may sometimes fail to produce the structured output as requested, instead returning something else, such as JSON with additional comments. The larger the model, the more consistently it returns the expected output. In this example, choosing `GPT_4_O` over `GPT_4_O_MINI` results in the correct movie choice ('Driving Miss Daisy') as well as properly formatted JSON. In real-life applications, consider implementing a validation mechanism to ensure the model's output is in the correct format.

### Using Tools

Tools allow LLMs  access your custom services in a powerful and flexible way. Let's use tools to work with OpenAI's function calling, and implement a weather service query.

Without additional tools, the model won't provide any information about the current weather, saying that it's unable to provide real-time weather updates:


In [ ]:
chatModel.call("What's the weather like in Paris today?")

Let's imagine we have a weather service providing weather information for different locations. Using tools, we can equip OpenAI with the access to this service. In this tutorial, we'll use `mockWeatherService` to mock such a service:

In [ ]:
fun mockWeatherService(location: String): Double? = when {
    "Paris" in location -> 15.0
    "Tokyo" in location -> 10.0
    "San Francisco" in location -> 30.0
    else -> null
}

We need to provide the model access to the weather tool. First, we define a `FunctionTool` with the name `"getCurrentWeather"` and the description `"Get current temperature for a given location."`:

In [ ]:
import org.springframework.ai.model.ModelOptionsUtils

val functionTool = FunctionTool(
    FunctionTool.Type.FUNCTION,
    FunctionTool.Function(
        "Get current temperature for a given location.",
        "getCurrentWeather", ModelOptionsUtils.jsonToMap(
            """
                {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City and country e.g. Bogotá, Colombia"
                        }
                    },
                    "required": ["location"],
                    "additionalProperties": false
                }
                """.trimIndent()
        ),
        true
    )
)

Now we send the user question together with the list of available tools:

In [ ]:
import org.springframework.ai.openai.api.OpenAiApi.*
import org.springframework.ai.openai.api.OpenAiApi.ChatCompletionRequest.ToolChoiceBuilder

val initialUserMessage = ChatCompletionMessage(
    "What's the weather like in Paris today?",
    ChatCompletionMessage.Role.USER
)
val chatCompletionRequest = ChatCompletionRequest(
    listOf(initialUserMessage), "gpt-4o",
    listOf(functionTool), ToolChoiceBuilder.AUTO
)

Depending on the user question, the model can now return the response with the information about the tools it decides to use and the arguments to provide to these tools. If the user asks about the weather, the model decides to use our weather tool. If the user asks an unrelated question, the model behaves as usual. We can display the whole response to check the chosen tools:

In [ ]:
val chatCompletion = openAiApi.chatCompletionEntity(chatCompletionRequest)
val responseFromLLM = chatCompletion.body!!.choices().first().message()
responseFromLLM

The response specifies the tool the LLM wants to call and its arguments:

```function=ChatCompletionFunction[name=getCurrentWeather, arguments={"location":"Paris, France"}]```

We invoke the tool and send the result back to the model, so that it could prepare the final response to the user (or, probably, decide to call other tools depending on the conversation):

In [ ]:
lateinit var messageWithToolInvocation: ChatCompletionMessage
for (toolCall in responseFromLLM.toolCalls()) {
    when (val functionName = toolCall.function().name()) {
        "getCurrentWeather" -> {
            val location = toolCall.function().arguments()
            val temperature = mockWeatherService(location)
            messageWithToolInvocation = ChatCompletionMessage(
                if (temperature != null) "$temperature C" else "Unable to get the weather",
                ChatCompletionMessage.Role.TOOL,
                functionName, toolCall.id(), null, null, null
            )
        }
    }
}

Now we send all the messages to LLM to provide the full context: the initial message, the response with the tool choice and the tool invocation result. Given this information, LLM can now answer the initial user question about the current weather in Paris:

In [ ]:
val messages = mutableListOf(initialUserMessage, responseFromLLM, messageWithToolInvocation)
val functionResponseRequest = ChatCompletionRequest(messages, "gpt-4o", 0.2)
val resultingCompletion = openAiApi.chatCompletionEntity(functionResponseRequest)
resultingCompletion.body!!.choices().first().message().content()

LLM was able to use the provided tool to respond to the user. Enhancing LLMs with external tools can automate tasks like data retrieval, customer support, and IoT control.

This notebook serves as an overview of how to integrate Spring AI into your Kotlin projects, empowering you to create powerful AI-driven applications. Experiment further with prompts and tailored implementations for your specific needs! 🚀